<a href="https://colab.research.google.com/github/vikysena9-design/python-ai-Rudnikova-Viktoria/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- `cartoons_capital_cost.csv` — бюджеты мультфильмов
- `cartoons_genre_country_duration.csv` — жанры, страны и продолжительность мультфильмов

**Что мы делаем:**
1. Клонируем репозиторий GitHub в Colab
2. Читаем CSV-файлы в pandas DataFrame
3. Очищаем и переименовываем столбцы
4. Смотрим структуру данных и делаем быструю валидацию

## 🐱 [1] Клонируем репозиторий курса в Colab

In [ ]:
# 🐱 Шаг 1. Клонируем репозиторий курса в Colab

import os

if not os.path.exists("python-ai-template"):
    !git clone -q https://github.com/componavt/python-ai-template.git

%cd python-ai-template

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-template")

## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [ ]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

df_cost = pd.read_csv("data/examples/cartoons_capital_cost.csv")
df_genre = pd.read_csv("data/examples/cartoons_genre_country_duration.csv")

print("✅ Загружено строк в df_cost:", len(df_cost))
print("✅ Загружено строк в df_genre:", len(df_genre))

## 🧹 [2B] Очистка и переименование столбцов

В исходных CSV-файлах есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `film` с URL (ссылкой на объект Wikidata) — нам не нужна ссылка, нам достаточно названия фильма.
- Столбцы `filmLabel`, `genreLabel`, `countryLabel` содержат читаемые подписи (название фильма, жанр, страна).

В этом шаге мы:
- удалим столбцы с URL Wikidata;
- переименуем `filmLabel → film`, `genreLabel → genre`, `countryLabel → country`;
- приведём числовые столбцы (`capital_cost`, `duration`) к типу `int`.

При приведении к числам мы используем:

- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(int)` — переводит столбец к целочисленному типу.

> ⚠️ **Важно:** если в ваших данных есть столбцы с URL Wikidata и столбцы вида `*Label`, этот шаг обязателен, чтобы получить аккуратные таблички для анализа.

In [ ]:
# 🧹 Шаг 2B. Очистка и переименование столбцов

# 1) df_cost: бюджеты мультфильмов
df_cost = df_cost.drop(columns=["film"])  # URL Wikidata нам не нужен
df_cost = df_cost.rename(columns={"filmLabel": "film"})
df_cost["capital_cost"] = pd.to_numeric(
    df_cost["capital_cost"], errors="coerce"
).fillna(0).astype(int)

# 2) df_genre: жанры, страны, продолжительность
df_genre = df_genre.drop(columns=["film"])
df_genre = df_genre.rename(columns={
    "filmLabel": "film",
    "genreLabel": "genre",
    "countryLabel": "country",
})
df_genre["duration"] = pd.to_numeric(
    df_genre["duration"], errors="coerce"
).fillna(0).astype(int)

print("✅ Данные очищены и готовы к анализу")

## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по бюджету (`capital_cost`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код два раза.

In [ ]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_cost, "Бюджеты мультфильмов (df_cost)")
show_info(df_genre, "Жанры, страны и продолжительность (df_genre)")

print("\n📈 Статистика по бюджету (capital_cost):")
print(df_cost["capital_cost"].describe())

In [ ]:
# То же самое, но в миллионах долларов (млн $)
stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
print("\n📈 Статистика по бюджету в млн $:")
print(stats_millions)

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** фильмов, стран и жанров есть в данных;
- **какие страны встречаются чаще всего** (Топ‑5 по числу строк);
- **какие жанры самые популярные** (Топ‑10 по числу строк).

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому  
`df_genre["country"].value_counts().head()` даёт **Топ‑5 стран по числу записей**.

In [ ]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔍 Быстрая проверка данных")

# Датасет 1: бюджеты
print("\nУникальных мультфильмов в df_cost:", df_cost["film"].nunique())
print("Диапазон бюджетов (млн $):",
      df_cost["capital_cost"].min() / 1e6, "—", df_cost["capital_cost"].max() / 1e6)

# Датасет 2: жанры, страны, длительность
print("\nУникальных мультфильмов в df_genre:", df_genre["film"].nunique())
print("Уникальных стран:", df_genre["country"].nunique())
print("Уникальных жанров:", df_genre["genre"].nunique())

print("\nТоп-5 стран по числу записей:")
print(df_genre["country"].value_counts().head())

print("\nТоп-10 жанров:")
print(df_genre["genre"].value_counts().head(10))

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨